$\newcommand{\To}{\Rightarrow}$

In [1]:
import os
os.chdir('..')

In [2]:
from kernel.type import STVar, TVar, TFun, BoolType, NatType
from kernel.term import Term, Var, SVar, Const, Eq, Lambda, Nat, Or, And, Forall, NatVars, Inst
from data import nat
from logic import basic
from logic import matcher

basic.load_theory('nat')

## Patterns and substitution

A pattern is a term containing schematic (term) variables. Substitution on terms is analogous to substitution on types, but more complicated due to the presence of abstractions (lambda terms), and the need to substitute for both type and term variables. Consider the following term:

In [3]:
Ta = STVar("a")
a = SVar("a", Ta)
b = SVar("b", Ta)
t = Eq(a, b)
print(t)
print(repr(t))

?a = ?b
Comb(Comb(Const(equals, ?'a => ?'a => bool), SVar(a, ?'a)), SVar(b, ?'a))


Observe that there are both type and term variables in $t$. The method `subst_type` takes a dictionary of assignments for type variables, and substitutes using this dictionary.

In [4]:
t2 = t.subst_type(a=NatType)
print(t2)
print(repr(t2))

?a = ?b
Comb(Comb(Const(equals, nat => nat => bool), SVar(a, nat)), SVar(b, nat))


Next, we can apply the method `subst`, which takes a dictionary of assignments for (term) variables, and substitutes using this dictionary.

In [5]:
t3 = t2.subst(a=nat.zero, b=nat.one)
print(t3)
print(repr(t3))

(0::nat) = 1
Comb(Comb(Const(equals, nat => nat => bool), Const(zero, nat)), Const(one, nat))


Actually, we can apply `subst` directly, which automatically figures out the type instantiation for us:

In [6]:
t4 = t.subst(a=nat.zero, b=nat.one)
print(t4)

(0::nat) = 1


We now look at some examples demonstrating how substitution interacts with abstractions. Consider the following term $t$:

In [7]:
x = SVar("x", NatType)
y = SVar("y", NatType)
t = Lambda(x, x + y)(nat.one) + x
print(t)

(%x. x + ?y) 1 + ?x


This term contains variables $x$ and $y$. Let's substitute 3 for $x$ and 5 for $y$:

In [8]:
t2 = t.subst(x=Nat(3), y=Nat(5))
print(t2)

(%x::nat. x + 5) 1 + 3


Observe that both the $x$ at the end and the $y$ inside the lambda term is substituted. However, the $x$ inside the lambda term is not. This is because the latter $x$ is a bound variable, and quite different from the former $x$. In fact, since the name of the bound variable does not matter, the term $t$ is equivalent to $(\lambda z. z + y) 1 + x$. The fact that the name of the bound variable is the same as a variable outside the lambda term is simply a coincidence.

By default, substitution does not perform $\beta$-conversion (evaluation of functions). For example:

In [9]:
f = SVar("f", TFun(NatType, NatType))
a = SVar("a", NatType)
t = f(a)
print('t: ', t)

t:  ?f ?a


In [10]:
x = Var('x', NatType)
t2 = t.subst(f=Lambda(x, x + 2), a=Nat(3))
print(t2)

(%x::nat. x + 2) 3


To evaluate $f$ after a substitution, one can use `beta_norm` function introduced previously. The function `subst_norm` combines the three operations above: type substitution, term substitution, and $\beta$-normalization. This function takes instantiations for term variables.

In [11]:
t3 = t.subst_norm(f=Lambda(x, x + 2), a=Nat(3))
print(t3)

(3::nat) + 2


We can also use `logic.subst_norm` to do the example at the beginning of the section:

In [12]:
Ta = STVar("a")
a = SVar("a", Ta)
b = SVar("b", Ta)
t = Eq(a, b)
t2 = t.subst_norm(a=nat.zero, b=nat.one)
print(t2)

(0::nat) = 1


## Matching

As with types, we can match a pattern (a term containing type and term variables) with a term. Matching a pattern $p$ with a term $t$ determines whether it is possible to instantiate $p$ to $t$, and if it is possible, produces the assignment of type and term variables. Matching of terms is complicated by the presence of two kinds of variables, as well as presence of abstractions (including the possibility of $\beta$-conversion). We start with the discussion with some simple cases, then gradually move to more complex situations.

The basic function for matching is `matcher.first_order_match`. It takes two arguments: the pattern and the term to be matched. It returns a pair (tyinst, inst), where tyinst is the dictionary of type instantiations, and inst is the dictionary of term instantiations. For example:

In [13]:
A = SVar("A", BoolType)
B = SVar("B", BoolType)
p = Or(A, B)
print('p:', p)

x = Var("x", NatType)
t = Or(Eq(x, 0), Eq(x, 1))
print('t:', t)

inst = matcher.first_order_match(p, t)
print(inst)

p: ?A | ?B
t: x = 0 | x = 1
A := x = 0, B := x = 1


The output says the term $x = 0 \vee x = 1$ matches the pattern $A \vee B$, where $A$ is assigned to $x = 0$ and $B$ is assigned to $x = 1$. If there is no match, the function `first_order_match` raises an exception. For example:

In [14]:
t2 = And(Eq(x, 0), Eq(x, 1))
print(t2)

matcher.first_order_match(p, t2)   # raises MatchException

x = 0 & x = 1


MatchException: [(Comb(Comb(Const(disj, bool => bool => bool), SVar(A, bool)), SVar(B, bool)), Comb(Comb(Const(conj, bool => bool => bool), Comb(Comb(Const(equals, nat => nat => bool), Var(x, nat)), Const(zero, nat))), Comb(Comb(Const(equals, nat => nat => bool), Var(x, nat)), Const(one, nat)))), (Comb(Const(disj, bool => bool => bool), SVar(A, bool)), Comb(Const(conj, bool => bool => bool), Comb(Comb(Const(equals, nat => nat => bool), Var(x, nat)), Const(zero, nat)))), (Const(disj, bool => bool => bool), Const(conj, bool => bool => bool))]

We now show an example with type variables.

In [15]:
Ta = STVar("a")
a = SVar("a", Ta)
b = SVar("b", Ta)
p = Eq(a, b)

t = Eq(nat.zero, nat.one)
inst = matcher.first_order_match(p, t)
print(inst)

'a := nat, a := (0::nat), b := (1::nat)


Matching can certainly go inside abstractions:

In [16]:
x = SVar("x", Ta)
P = SVar("P", TFun(Ta, BoolType))
p = Forall(x, P(x))
print('p:', p)

n = Var("n", NatType)
A = Var("A", TFun(NatType, BoolType))
t = Forall(n, A(n))
print('t:', t)

inst = matcher.first_order_match(p, t)
print(inst)

p: !x. ?P x
t: !n. A n
'a := nat, P := A


The previous matching worked because the body of the $\forall$ quantifier is precisely a function applied to the bound variable. However, matching still works if the body is a more general predicate of the bound variable:

In [17]:
t2 = Forall(n, Eq(n, 0))
print('t2:', t2)

inst = matcher.first_order_match(p, t2)
print(inst)

t2: !n::nat. n = 0
'a := nat, P := %x::nat. x = 0


In this case, $p$ can be transformed to $t$ only after instantiation as well as $\beta$-normalization. In general, `first_order_match` determines whether there exists `inst` such that `p.subst_norm(inst)` is equal to $t$.

## Preview: applying a theorem

Matching and substitution play crucial roles in proofs in higher-order logic, in particular when applying a theorem. We will start the discussion of theorems and proofs in the next section. Here, we give a brief preview of these ideas, using what we have learned so far.

One important class of theorems is identities. For example, the distributivity of multiplication over addition is stated as follows:

In [18]:
x = SVar("x", NatType)
y = SVar("y", NatType)
z = SVar("z", NatType)

distrib_l = Eq(x * (y + z), x * y + x * z)
print(distrib_l)

?x * (?y + ?z) = ?x * ?y + ?x * ?z


Here $x,y,z$ are variables that can represent arbitrary natural numbers. We would like to apply this identity to rewrite any expression of the form $x\cdot (y+z)$, for example $a\cdot (2a+(b+1))$, which should be rewritten to $a\cdot 2a+a(b+1)$.

The idea for implementing this functionality is as follows. Suppose the term to be rewritten is $t$. First match the left side of the equality with $t$. If the matching succeeds, use the resulting assignments of variables to instantiate the right side of the equality.

In [19]:
def rewrite(prop, t):
    """Rewrite term t using the identity (equality term) prop."""
    inst = matcher.first_order_match(prop.lhs, t)
    return prop.rhs.subst_norm(inst)

We now test this function on an example:

In [20]:
a = Var("a", NatType)
b = Var("b", NatType)
t = a * (2 * a + (b + 1))
print('Before:', t)

t2 = rewrite(distrib_l, t)
print("After: ", t2)

Before: a * (2 * a + (b + 1))
After:  a * (2 * a) + a * (b + 1)


We have written our first program for proof automation! But before we get too excited - so far all we are doing is manipulating terms. In the next section, we will start the discussion of theorems, which allows us to describe proofs in a more precise language. In particular, we will see how to make sure that the proofs we construct are correct, without trusting the proof automation we wrote ourselves.